# Cheat Sheet
Dies ist ein cheat sheet mit geläufigen Dingen für Pytorch und Tensorflow
## Pytorch
(Ein Einsteiger Tutorial für pytorch ist auch [hier](https://pytorch.org/tutorials/beginner/basics/intro.html) zu finden)

In [69]:
import torch
from torch import nn
import numpy as np

Der folgende Code abschnitt kann genutzt werden, um zu schauen, welche Hardware (für das Training) genutzt werden kann.

In [70]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


Um inputs an das neuronale Netz geben zu können, müssen diese zunächst als Tensoren gespeichert werden.

In [71]:
xor_inputs = torch.Tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])

# Alternative mit numpy
np_array = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
x_np = torch.from_numpy(np_array)

print(xor_inputs)
print(x_np)

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])
tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], dtype=torch.float64)


Ein neuronales Netz ist ein Modul, dessen Struktur aus Layer Modulen aufgebaut wird. Durch diese Struktur ist es möglich das neuronale Netz leicht so zu designen, wie man es haben möchte. <br/>
Bei ```nn.Flatten``` wird die Eingabe beliebiger Dimension in einen Tensor umgewandelt. Dies kann genutzt werden um Bilder in ein passendes Format zu bringen. <br/>
```nn.Sequential``` ist dafür da mehrere Module in der gegebenen Reihenfolge zu speichern. Die Daten werden in dieser Reihenfolge durch die Layer Module gegeben. <br/>
Bei ```nn.Linear``` ist der Layer vollständig Verbunden. <br/>
Die Aktivierungsfunktionen können als einzelne Layer behandelt werden, benötigen aber keine Dimension als Parameter. Man kann sie aber auch beim implementieren der Forward-Funktion aufrufen. Aktivierungsfunktionen können [hier](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity) gefunden werden. <br/>
<br/>
Es wird außerdem gezeigt, wie man Gewichte initaliseren kann.


In [72]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Sigmoid(),
            nn.Linear(512, 10)
        )

        # Initialize weights
        torch.nn.init.uniform_(self.linear_stack[0].weight, -1.0, 1.0)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_stack(x)
        return logits

Folgend wird festgelegt, welche Hardware für das neuronale Netz genutzt werden soll. Außerdem wird gezeigt wie man auf Inhalte des neuronalen Netzes zugreifen kann.

In [73]:
model = NeuralNetwork().to(device)

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0426, -0.0579,  0.5217,  ..., -0.9621, -0.5584, -0.5798],
        [-0.3622,  0.2837,  0.0736,  ..., -0.1286, -0.5135, -0.5226]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0353, 0.0085], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 1.3414e-03, -1.3428e-02, -3.4596e-02,  ..., -2.9251e-02,
          4.2676e-02,  8.4966e-03],
        [-2.6279e-05, -2.9826e-02,  3.4341e-03,  ...,  4.2719e-02,
         -3.9291e-02, -4.

Es gibt verschiedene Loss-Funktionen, die in der Library vorhanden sind. Außerdem gibt es verschiedene Optimizer, welche die Parameter, die optimiert werden sollen, sowie die learning rate als Parameter bekommen. (Es gibt noch weitere optionale Parameter.)

In [74]:
torch_loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Der folgende Abschnitt zeigt, wie man die Loss-Funktion und den Optimizer einsetzen kann.

In [80]:
def show(data):
    size = len(data)
    for batch, (X, y) in enumerate(data):
        # Compute prediction and loss
        pred = model(X)
        loss = torch_loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## Tensorflow
(Ein Einsteiger Tutorial für tensorflow ist auch [hier](https://www.tensorflow.org/tutorials/quickstart/beginner) zu finden)

In [76]:
import tensorflow as tf

Man kann diese Inputs einfach so an Tensorflow geben, man kann sie aber auch stattdessen in einen Tensor konvertieren.

In [86]:
# input X vector
X = [[0, 0], [0, 1], [1, 0], [1, 1]]

# output Y vector
Y = [[0], [1], [1], [0]]

print(tf.convert_to_tensor(X))

tf.Tensor(
[[0 0]
 [0 1]
 [1 0]
 [1 1]], shape=(4, 2), dtype=int32)


Die Struktur ist, wie bei Pytorch ein neuronales Netz, welches mehrere Layer enthält. Auch hier wird ```Sequential``` zum Stapeln von Layern genutzt.
```Flatten``` wird auch genutzt um die Dimension der Eingabe anzupassen.
Die voll vernetzten Layer werden hier ```Dense``` genannt. Ein unterschied zu Pytorch ist, dass die Aktivierunsfunktionen hier keine einzelnen Layer sind, sondern bei den Layern direkt mit angegeben werden können.

Es wird auch gezeigt, wie man die gewichte eines Layers initialisieren kann.

In [78]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, 
                        activation='relu',
                        kernel_initializer=tf.keras.initializers.RandomUniform(-1., 1.),
                        bias_initializer=tf.keras.initializers.RandomUniform(-1., 1.)),
  tf.keras.layers.Dense(10, activation='sigmoid')
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 784)               0         
                                                                 
 dense_12 (Dense)            (None, 128)               100480    
                                                                 
 dense_13 (Dense)            (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


Der folgende Abschnitt zeigt, wie man den Optimizer und die Loss Funktion nutzen kann.

In [79]:
tf_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=tf_loss_fn,
              metrics=['accuracy'])
